In [4]:
# Import required libraries
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

# Groq model initialization
GROQ_API_KEY = "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"
model = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama-3.2-3b-preview",
)

In [5]:
# Tools for AD Agent
def assign_user_to_sg(user: str, sg: str) -> str:
    """Assign a user to a security group."""
    return f"Assigned user {user} to security group {sg}"

def validate_user(user: str) -> str:
    """Validate a user in Active Directory."""
    return f"Validated user {user}"

def construct_upn(user: str) -> str:
    """Construct a User Principal Name (UPN)."""
    return f"Constructed UPN for {user}: {user}@domain.com"

# Tools for M365 Agent
def create_dl(dl_name: str) -> str:
    """Create a distribution list in Microsoft 365."""
    return f"Created distribution list {dl_name}"

def delete_dl(dl_name: str) -> str:
    """Delete a distribution list in Microsoft 365."""
    return f"Deleted distribution list {dl_name}"

def assign_license(user: str, license: str) -> str:
    """Assign a license to a user in Microsoft 365."""
    return f"Assigned {license} license to {user}"

def create_shared_mailbox(mailbox_name: str) -> str:
    """Create a shared mailbox in Microsoft 365."""
    return f"Created shared mailbox {mailbox_name}"

def modify_shared_mailbox(mailbox_name: str, action: str) -> str:
    """Modify a shared mailbox in Microsoft 365."""
    return f"Modified shared mailbox {mailbox_name} with action {action}"

# Tools for DB Restart Agent
def restart_db(db_name: str) -> str:
    """Restart a database."""
    return f"Restarted database {db_name}"

In [6]:

# Create AD Agent
ad_agent = create_react_agent(
    model=model,
    tools=[assign_user_to_sg, validate_user, construct_upn],
    name="ad_expert",
    prompt="You are an Active Directory expert. Use the provided tools to handle AD-related tasks such as assigning users to security groups, validating users, or constructing UPNs. Use one tool at a time."
)

# Create M365 Agent
m365_agent = create_react_agent(
    model=model,
    tools=[create_dl, delete_dl, assign_license, create_shared_mailbox, modify_shared_mailbox],
    name="m365_expert",
    prompt="You are a Microsoft 365 expert. Use the provided tools to handle M365-related tasks such as creating or deleting distribution lists, assigning licenses, or managing shared mailboxes. Use one tool at a time."
)

# Create DB Restart Agent
db_restart_agent = create_react_agent(
    model=model,
    tools=[restart_db],
    name="db_restart_expert",
    prompt="You are a database management expert. Use the provided tools to handle database-related tasks such as restarting a database. Use one tool at a time."
)

In [7]:
from langgraph_supervisor import create_supervisor

# Create supervisor workflow
workflow = create_supervisor(
    [ad_agent, m365_agent, db_restart_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing an Active Directory expert (ad_expert), a Microsoft 365 expert (m365_expert), and a database management expert (db_restart_expert). "
        "Route queries to the appropriate agent based on the task: "
        "For Active Directory tasks (e.g., user assignment, validation, UPN construction), use ad_expert. "
        "For Microsoft 365 tasks (e.g., distribution lists, licenses, shared mailboxes), use m365_expert. "
        "For database tasks (e.g., restarting a database), use db_restart_expert."
    )
)

In [11]:
# Compile the workflow
app = workflow.compile()

# Invoke the workflow with a query
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Assign user to SG"
        }
    ]
})

# Print the result
print(result["messages"][-1].content)

<function=transfer_to_ad_expert>{"username": "example_user", "groupname": "SG"}
